## Project: Search and Sample Return by Edwin Montgomery

---


**The goals / steps of this project are the following:**  

**Training / Calibration**  

* Download the simulator and take data in "Training Mode"
* Test out the functions in the Jupyter Notebook provided
* Add functions to detect obstacles and samples of interest (golden rocks)
* Fill in the `process_image()` function with the appropriate image processing steps (perspective transform, color threshold etc.) to get from raw images to a map.  The `output_image` you create in this step should demonstrate that your mapping pipeline works.
* Use `moviepy` to process the images in your saved dataset with the `process_image()` function.  Include the video you produce as part of your submission.

**Autonomous Navigation / Mapping**

* Fill in the `perception_step()` function within the `perception.py` script with the appropriate image processing functions to create a map and update `Rover()` data (similar to what you did with `process_image()` in the notebook). 
* Fill in the `decision_step()` function within the `decision.py` script with conditional statements that take into consideration the outputs of the `perception_step()` in deciding how to issue throttle, brake and steering commands. 
* Iterate on your perception and decision function until your rover does a reasonable (need to define metric) job of navigating and mapping.  

[//]: # (Image References)

[image1]: ./misc/rover_image.jpg
[image2]: ./calibration_images/example_grid1.jpg
[image3]: ./calibration_images/example_rock1.jpg 
[image4]: ./code/autonomous_mapping.png
[image5]: ./misc/navigable5.png
[image5A]: ./misc/thresh_rock.png
[image6]: ./misc/perspective.png
[image7]: ./misc/warp.png
[image8]: ./misc/MapToWorld.png
[image9]: ./misc/HSVthreshold.png




## [Rubric](https://review.udacity.com/#!/rubrics/916/view) Points
### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---
### Writeup / README


#### Simulator / Environment Setup

* MacBook Pro Running OS Sierra  10.12  
* Installed  Anaconda and Jupyter Notebooks per the instructions.
* used PIP to install moviepy 
* Downloaded MacOSX version Robo Simulator 2. The initial Robot Simulator didn't work well in automonous mode.
* Forked a version of RoboND-Rover Rover to my  ejm2000 GitHub account.

My Github account with the entire project and writeup:
(https://github.com/ejm2000/RoboND-Rover-Project) 
         
       
        
#### The Perception Step

Modified a function to test  each channel R,G,B for a given pixel against a set RGB threhold. The color thresold function is set to (!60,160,160) which corresponds to the lighter color pixels which corresponds to navigable terrain.
The code snippet and figures from before and after calling the color threshold function are below

        def color_thresh(img, rgb_thresh=(160, 160, 160)):
            # Create an array of zeros same xy size as img, but single channel
            color_select = np.zeros_like(img[:,:,0])
            # Require that each pixel be above all three threshold values in RGB
            # above_thresh will now contain a boolean array with "True"
            # where threshold was met
            above_thresh = (img[:,:,0] > rgb_thresh[0]) \
                        & (img[:,:,1] > rgb_thresh[1]) \
                        & (img[:,:,2] > rgb_thresh[2])
            # Index the array of zeros with the boolean array and set to 1
            color_select[above_thresh] = 1
            # Return the binary image
            return color_select

![Detectable Navigable Terrain before and after][image5]




I wrote a custom the color ranging function to detect rock samples. The rock samples are yellow , so set range to detect rocks in that range.  Modifed the color_thresh function to take a upper and lower rnage of RBG values.

       def color_range(img, rgb_thresh,object_thresh):
            # Create an array of zeros same xy size as img, but single channel
            color_select = np.zeros_like(img[:,:,0])
            # Require that each pixel be above and below all three threshold values in RGB
            # above_thresh & below_thresh will now contain a boolean array with "True"
            # where threshold was met
            above_thresh = (img[:,:,0] > rgb_thresh[0]) \
                    & (img[:,:,1] > rgb_thresh[1]) \
                    & (img[:,:,2] > rgb_thresh[2])

            below_thresh = (img[:,:,0] < object_thresh[0]) \
                    & (img[:,:,1] < object_thresh[1]) \
                    & (img[:,:,2] < object_thresh[2])

            # Index the array of zeros with the boolean array and set to 1 
            # when the values are in range

            color_select[(above_thresh & below_thresh)] = 1

            # Return the binary image
            return color_select                                     
                            
                            

![Detectable Rocks][image5A]


One  can also use openCv function called cvtColor  find upper and lower rangr of BGR  values and mask them to isolate the the target object. In the example below the function is used to find threshold for the rock samples.

HSV colorspaces can be tricky to determine so I used the following utility to test out different color values
* <http://www.rapidtables.com/web/color/color-picker.htm>

The code and screenshots below demonstrates the process:

        import cv2 # OpenCV for perspective transform
        import numpy as np
        example_rock = '../calibration_images/example_rock1.jpg'

        rock_img = cv2.imread(example_rock)

        # define range of yellow color in HSV
        hsv = cv2.cvtColor(rock_img, cv2.COLOR_BGR2HSV)
        lower_yellow = np.array([0,100,100])
        upper_yellow = np.array([176,255,255])

        # Threshold the HSV image to get only yellow colors
        mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        # Bitwise-AND mask and original image
        result = cv2.bitwise_and(rock_img,hsv, mask= mask)


        cv2.imshow('camera image of rock',rock_img)
        cv2.imshow('mask',mask)
        cv2.imshow('result',result)


        cv2.waitKey(30000)
        cv2.destroyAllWindows()                            


![Using HSV color spaces to detect objects][image9]


#### Perspective Transform

Applied OpenCv functions to incoming Camera Image to convert perspective view to overhead view for mapping navigable terrain. Chose pixels from a perspective camera image blockand mapped them the associated pixels in an overhead view. 

Code excerpt:

    source = np.float32([[10,140  ], [118,95], [200,95 ], [ 300,140]])
    destination = np.float32([[ 150,150 ], [150 ,140], [160,140 ], [160,150 ]])      
    warped = perspect_transform(image, source, destination)


    def perspect_transform(img, src, dst):

        # Get transform matrix using cv2.getPerspectivTransform()
        M = cv2.getPerspectiveTransform(src, dst)
        # Warp image using cv2.warpPerspective()
        # keep same size as input image
        warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))
        # Return the result
        return warped
    
![Perspective][image6]

#### Warp, Threshold

Converted Map to Rover Centric Coordinates where the Camera is set to (0,0) on the coorindate plane

    def rover_coords(binary_img):
        # Calculate pixel positions with reference to the rover 
        # position being at the center bottom of the image.
        ypos, xpos = colorsel.nonzero()
        y_pixel = -(ypos - binary_img.shape[0]).astype(np.float)
        x_pixel = -(xpos - binary_img.shape[1]/2).astype(np.float)
        return x_pixel, y_pixel
        
![Rover Coordinates][image7]        

####  Map to World Coordinates

Applied a Rotational Matrix on the Rover's Position to get the yaw, then scaled & mapped it to the world coordinators.

    # Define a function to apply a rotation to pixel positions
    def rotate_pix(xpix, ypix, yaw):
        # TODO:Done
        # Convert yaw to radians
        # Apply a rotation ,please
        yaw_rad = yaw * np.pi / 180
        xpix_rotated = xpix * np.cos(yaw_rad) - ypix * np.sin(yaw_rad)
        ypix_rotated = xpix * np.sin(yaw_rad) + ypix * np.cos(yaw_rad)
        # Return the rotated result  
        return xpix_rotated, ypix_rotated

    # Define a function to perform a translation
    def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
        # TODO:Done
        # Apply a scaling and a translation
        # Perform translation and convert to integer since pixel values can't be float
        xpix_translated = np.int_(xpos + (xpix_rot / scale))
        ypix_translated = np.int_(ypos + (ypix_rot / scale))
        # Return the result  
        return xpix_translated, ypix_translated

    def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
        # Apply rotation
        xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
        # Apply translation
        xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
        # Perform rotation, translation and clipping all at once
        x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
        y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
        # Return the result
        return x_pix_world, y_pix_world
        
![Map to Word][image8]        

#### Decisions and Taking Action

Convert the x,y coordinates to polar coordinates to calculate the average angle to steer the rover.

    Rover.nav_dists, Rover.nav_angles = to_polar_coords(xpix, ypix)
    
    def to_polar_coords(x_pixel, y_pixel):
        # Convert (x_pixel, y_pixel) to (distance, angle) 
        # in polar coordinates in rover space
        # Calculate distance to each pixel
        dist = np.sqrt(x_pixel**2 + y_pixel**2)
        # Calculate angle away from vertical for each pixel
        angles = np.arctan2(y_pixel, x_pixel)
        return dist, angles


The Rover State is maintained in the Rover class. This class has attributes such as x,y position, speed, velocity, incoming camera images, yaw, etc.

This information will serve as the basis for Rover control , decision making , and navigation.


### Notebook Analysis
#### 1. Run the functions provided in the notebook on test images (first with the test data provided, next on data you have recorded). Add/modify functions to allow for color selection of obstacles and rock samples. Describe in your writeup (and identify where in your code) how you modified or added functions to add obstacle and rock sample identification.

Link to Rover Project Test NoteBook:
* <http://localhost:8888/notebooks/code/Rover_Project_Test_Notebook.ipynb>

 
I created a color_range thresholding function to detect rocks witin a certain color range. The sample rocks were in the yellow range of colors. It is similar to the function color_thresh which detects colors above a certain threshold


                # Identify pixels above the threshold
                # Threshold of RGB > 160 does a nice job of identifying ground pixels only
                # Yellow is 255,255,0
                # 
                def color_thresh(img, rgb_thresh=(160, 160, 160)):
                    # Create an array of zeros same xy size as img, but single channel
                    color_select = np.zeros_like(img[:,:,0])
                    # Require that each pixel be above all three threshold values in RGB
                    # above_thresh will now contain a boolean array with "True"
                    # where threshold was met
                    above_thresh = (img[:,:,0] > rgb_thresh[0]) \
                                & (img[:,:,1] > rgb_thresh[1]) \
                                & (img[:,:,2] > rgb_thresh[2])
                    # Index the array of zeros with the boolean array and set to 1
                    color_select[above_thresh] = 1
                    # Return the binary image
                    return color_select


                def color_range(img, rgb_thresh,object_thresh):
                    # Create an array of zeros same xy size as img, but single channel
                    color_select = np.zeros_like(img[:,:,0])
                    # Require that each pixel be above and below all three threshold values in RGB
                    # above_thresh & below_thresh will now contain a boolean array with "True"
                    # where threshold was met
                    above_thresh = (img[:,:,0] > rgb_thresh[0]) \
                                & (img[:,:,1] > rgb_thresh[1]) \
                                & (img[:,:,2] > rgb_thresh[2])

                    below_thresh = (img[:,:,0] < object_thresh[0]) \
                                & (img[:,:,1] < object_thresh[1]) \
                                & (img[:,:,2] < object_thresh[2])
                    # Index the array of zeros with the boolean array and set to 1 when the values are in range
                    color_select[(above_thresh & below_thresh)] = 1
                    # Return the binary image
                    return color_select

                threshed = color_thresh(warped)
                fig = plt.figure(figsize=(12,3))
                plt.subplot(121)
                plt.imshow(threshed, cmap='gray')

                #Plot Object 
                #rgb_thresh = (160,160,160)  #navigable terrain
                #object_thresh = (255,255,255)
                #works for detecting just rocks
                rgb_thresh = (170,134,0)  #rocks
                object_thresh = (255,170,70)
                threshed_rock = color_range(rock_warped,rgb_thresh,object_thresh)
                plt.subplot(122)
                plt.imshow(threshed_rock, cmap='gray')



#### 2. Populate the `process_image()` function with the appropriate analysis steps to map pixels identifying navigable terrain, obstacles and rock samples into a worldmap.  Run `process_image()` on your test data using the `moviepy` functions provided to create video output of your result. Describe in your writeup how you modified the process_image() to demonstrate your analysis and how you created a worldmap. Include your video output with your submission.

Link to Rover Project Test NoteBook:
* <http://localhost:8888/notebooks/code/Rover_Project_Test_Notebook.ipynb>

---
# The movie output of this code included in the link below:
# * <http://localhost:8888/notebooks/output/test_mapping.mp4>

---
    
Modified the calls to color_threshold & color_range functions to determine navigable objects, rocks and obstacles. 
As suggested in the Hints section, used an inverted navigable image to determine location of obstacles.

Create function to map rover coordinators to map coordinates, and called the "get_world_coordinates" function for navigable , rock, and obstacle coordinates and mapped them to the data objects for proper display in the movie. An excerpt from the code is shown below




        def get_world_coordinates(threshed):
            scale = 10
            # 4) Convert thresholded image pixel values to rover-centric coords
            xpix, ypix = rover_coords(threshed)
            # 5) Convert rover-centric pixel values to world coords

            x_world, y_world = pix_to_world(xpix, ypix, np.float32(data.xpos[data.count]), 
                                        np.float32(data.ypos[data.count]), np.float32(data.yaw[data.count]), 
                                        data.worldmap.shape[0], scale)

            return (x_world,y_world)

        # Define a function to pass stored images to
        # reading rover position and yaw angle from csv file
        # This function will be used by moviepy to create an output video
        def process_image(img):
            # Example of how to use the Databucket() object defined above
            # to print the current x, y and yaw values 
            # print ("Data xpos,ypos,yaw")
            # print(data.xpos[data.count], data.ypos[data.count], data.yaw[data.count])

            # TODO: Done
            # 1) Define source and destination points for perspective transform
            dst_size = 5 
            bottom_offset = 6
            scale = 10
            source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])

            destination = np.float32([[img.shape[1]/2 - dst_size, img.shape[0] - bottom_offset],
                          [img.shape[1]/2 + dst_size, img.shape[0] - bottom_offset],
                          [img.shape[1]/2 + dst_size, img.shape[0] - 2*dst_size - bottom_offset], 
                          [img.shape[1]/2 - dst_size, img.shape[0] - 2*dst_size - bottom_offset],
                          ])
            # 2) Apply perspective transform
            warped = perspect_transform(img, source, destination)

            # 3) Apply color thresholds to identify navigable terrain/obstacles/rock samples
            threshed = color_thresh(warped)
            threshed_obstacle = np.invert(threshed)  #inverse of navigable path are obstables

            rgb_thresh = (170,134,0)  #rocks
            object_thresh = (255,170,70)
            threshed_rock = color_range(warped,rgb_thresh,object_thresh)

            # 4) Convert thresholded image pixel values to rover-centric coords
            # 5) Convert rover-centric pixel values to world coords
            navigable_x_world,navigable_y_world = get_world_coordinates(threshed)
            rock_x_world,rock_y_world = get_world_coordinates(threshed_rock)
            obstacle_x_world,obstacle_y_world = get_world_coordinates(threshed_obstacle)



            # 6) Update worldmap (to be displayed on right side of screen)
                # Example: data.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
                #          data.worldmap[rock_y_world, rock_x_world, 1] += 1
                #          data.worldmap[navigable_y_world, navigable_x_world, 2] += 1

            #Navigable World
            data.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
            data.worldmap[rock_y_world, rock_x_world, 1] += 1
            data.worldmap[navigable_y_world, navigable_x_world, 2] += 1



![alt text][image2]

### Autonomous Navigation and Mapping

#### 1. Fill in the `perception_step()` (at the bottom of the `perception.py` script) and `decision_step()` (in `decision.py`) functions in the autonomous mapping scripts and an explanation is provided in the writeup of how and why these functions were modified as they were.

### perception_step() and decision_step() functions have been filled in and their functionality explained in the writeup.

The perception_step was completed in a similar manner as the process_image() function described in the NoteBook Analysis. The primary difference being that  outputs were mapped to the Rover State Object. In addition the Rover nav_distance and angles were updated using the  "to_polar_coordinates" function

        def get_world_coordinates(threshed,Rover):
            scale = 10
            # 5) Convert map image pixel values to rover-centric coords
            xpix, ypix = rover_coords(threshed)    
            # 6) Convert rover-centric pixel values to world coordinates

            x_world, y_world = pix_to_world(xpix, ypix, np.float32(Rover.pos[0]), 
                                        np.float32(Rover.pos[1]), np.float32(Rover.yaw), 
                                        Rover.worldmap.shape[0], scale)

            return (x_world,y_world)



        # Apply the above functions in succession and update the Rover state accordingly
        def perception_step(Rover):
            # Perform perception steps to update Rover()
            # TODO: Done
            # NOTE: camera image is coming to you in Rover.img
            # 1) Define source and destination points for perspective transform
            dst_size = 5 
            bottom_offset = 6
            scale = 10
            source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
            destination = np.float32([[Rover.img.shape[1]/2 - dst_size, Rover.img.shape[0] - bottom_offset],
                          [Rover.img.shape[1]/2 + dst_size, Rover.img.shape[0] - bottom_offset],
                          [Rover.img.shape[1]/2 + dst_size, Rover.img.shape[0] - 2*dst_size - bottom_offset], 
                          [Rover.img.shape[1]/2 - dst_size, Rover.img.shape[0] - 2*dst_size - bottom_offset],
                          ])
            # 2) Apply perspective transform
            warped = perspect_transform(Rover.img, source, destination)
            # 3) Apply color threshold to identify navigable terrain/obstacles/rock samples
            threshed = color_thresh(warped)

            threshed_obstacle = np.invert(threshed)  #inverse of navigable path are obstables

            rgb_thresh = (170,134,0)  #rocks
            object_thresh = (255,170,70)
            threshed_rock = color_range(warped,rgb_thresh,object_thresh)


            # 4) Update Rover.vision_image (this will be displayed on left side of screen)
                # Example: Rover.vision_image[:,:,0] = obstacle color-thresholded binary image
                #          Rover.vision_image[:,:,1] = rock_sample color-thresholded binary image
                #          Rover.vision_image[:,:,2] = navigable terrain color-thresholded binary image

            Rover.vision_image[:,:,0] = threshed_obstacle
            Rover.vision_image[:,:,1] = threshed_rock
            Rover.vision_image[:,:,2] = threshed*255

            # 5) Convert map image pixel values to rover-centric coords
            # 6) Convert rover-centric pixel values to world coordinates
            navigable_x_world,navigable_y_world = get_world_coordinates(threshed,Rover)
            rock_x_world,rock_y_world = get_world_coordinates(threshed_rock,Rover)
            obstacle_x_world,obstacle_y_world = get_world_coordinates(threshed_obstacle,Rover)


            # 7) Update Rover worldmap (to be displayed on right side of screen)
                # Example: Rover.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
                #          Rover.worldmap[rock_y_world, rock_x_world, 1] += 1
                #          Rover.worldmap[navigable_y_world, navigable_x_world, 2] += 1

            Rover.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
            Rover.worldmap[rock_y_world, rock_x_world, 1] += 1
            Rover.worldmap[navigable_y_world, navigable_x_world, 2] += 1

            # 8) Convert rover-centric pixel positions to polar coordinates
            # Update Rover pixel distances and angles
                # Rover.nav_dists = rover_centric_pixel_distances
                # Rover.nav_angles = rover_centric_angles

            xpix, ypix = rover_coords(threshed)
            Rover.nav_dists, Rover.nav_angles = to_polar_coords(xpix,ypix)


            return Rover

These nav angles were fed into the Decision_step


#### 2. Launching in autonomous mode your rover can navigate and map autonomously.  By running drive_rover.py and launching the simulator in autonomous mode, your rover does a reasonably good job at mapping the environment. Explain your results and how you might improve them in your writeup.  

#### The rover must map at least 40% of the environment with 60% fidelity (accuracy) against the ground truth. You must also find (map) the location of at least one rock sample. They don't need to pick any rocks up, just have them appear in the map (should happen automatically if their map pixels in Rover.worldmap[:,:,1] overlap with sample locations.)

                       
 Here I'll talk about the approach I took, what techniques I used, what worked and why, where the pipeline might fail and how I might improve it if I were going to pursue this project further.  
 
Simulator Settings:
* Screen resolution set to 1280 x 800. Graphics Quality settings: Good
               
 
With the basic perception_step modifications described in Step #1, the rover mapped pretty well but tended to crash into obstacles that were located the center of the path. 

After updating it with more obstacle mapping information 
* Performance improved with 68% mapping and 67% fidelity in 200+ seconds.

Autonomous Mapping ScreenShot from Simulator:
![Autonomous Mapping ScreenShot][image4] 



Code Snippet from perception.py:

            navigable_x_world,navigable_y_world = get_world_coordinates(threshed,Rover)
            rock_x_world,rock_y_world = get_world_coordinates(threshed_rock,Rover)
            obstacle_x_world,obstacle_y_world = get_world_coordinates(threshed_obstacle,Rover)

            # 7) Update Rover worldmap (to be displayed on right side of screen)
                
            Rover.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
            Rover.worldmap[rock_y_world, rock_x_world, 1] += 1
            Rover.worldmap[navigable_y_world, navigable_x_world, 2] += 1

            # 8) Convert rover-centric pixel positions to polar coordinates              
            xpix, ypix = rover_coords(threshed)
            Rover.nav_dists, Rover.nav_angles = to_polar_coords(xpix,ypix)                           



Unfortunately time did not allow for me to try the activities listed in challenge.
      

